In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
import numpy as np
import math



In [3]:
columnts = ['id_user', 'id_item', 'ratings', 'timestamp']

df= pd.read_csv('u.data', sep ='\t', header = None, names = columnts, index_col = False)


X_train, X_test= train_test_split(df, test_size=0.1, random_state=42)
X_base, X_val, = train_test_split(X_train, test_size=0.1112, random_state=62)

Media_notas = df["ratings"].sum()/len(df)
row = np.max(df['id_user'])
col = np.max(df['id_item'])

X_base

,id_user,id_item,ratings,timestamp
54090,736,248,4,878709365
12888,323,713,4,878739299
92711,85,715,4,882995967
2231,160,187,5,876770168
57089,405,708,1,885546487
...,...,...,...,...
64995,429,972,4,882387757
55210,685,873,2,879451401
66460,896,76,3,887158359
50788,721,1392,3,877137598


In [4]:
def m_base(data):
  matriz = np.zeros((row, col), float)
  for i, info in data.iterrows():
    user = info.id_user -1;
    item = info.id_item -1;
    matriz[user][item] = info.ratings
  matriz
  return matriz

In [5]:
pam_calc = []
matriz = m_base(X_base)
matriz

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
def M_user(matriz):
  df_review = pd.DataFrame({"qtd_notas" : np.zeros(row, int),
                         "somatorio" : np.zeros(row, float)})
  for i in range(0, row):
    for j in range(0, col):
      if matriz[i][j] > 0:
        df_review.iloc[i,0] += 1
        df_review.iloc[i,1] += matriz[i][j]

  u_medias = df_review['somatorio']/ df_review['qtd_notas']

  return u_medias, df_review['qtd_notas']

In [7]:
medias, M_qtd = M_user(matriz)

In [8]:
def itens_comuns(u,v):
  itens = []
  sum = 0
  for i in range(0, len(u)):
    if u[i] != 0 and v[i] != 0:
      itens.append(i)

  return itens


# **Similaridade de Cosseno**

In [9]:
def calc_divisorCos(u,v):
  sum_u = 0
  sum_v = 0
  itens_u = []
  itens_v = []
  for i in range(0, len(u)):
    if u[i] != 0:
      itens_u.append(i)
    if v[i] != 0:
     itens_v.append(i)

  for j in itens_u:
      sum_u += u[j]**2
  for k in itens_v:
      sum_v += v[k]**2

  return sum_u*sum_v

def similaridade_cosseno(matriz):
  similaridade = np.zeros((row, row), float)

  for i in range(0,row):
    for j in range(i,row):
      if(i==j):
        similaridade[i,j] = 1
        continue

      sum = 0
      a = matriz[i,0:]
      b = matriz[j,0:]
      itens_c = itens_comuns(a,b)
      for k in itens_c:
       sum += matriz[i][k]*matriz[j][k]

      divisor = math.sqrt(calc_divisorCos(matriz[i,0:], matriz[j,0:]))
      similaridade[i][j] = sum/divisor
      similaridade[j][i] = similaridade[i][j]
  return similaridade



In [10]:
similar_a = similaridade_cosseno(matriz)

In [11]:
pam_calc.append(similar_a)
pam_calc[0]

array([[1.        , 0.12524512, 0.01675931, ..., 0.13183938, 0.09790009,
        0.31355682],
       [0.12524512, 1.        , 0.09461258, ..., 0.08821128, 0.16292235,
        0.05478139],
       [0.01675931, 0.09461258, 1.        , ..., 0.        , 0.12272133,
        0.01369132],
       ...,
       [0.13183938, 0.08821128, 0.        , ..., 1.        , 0.03394827,
        0.12493734],
       [0.09790009, 0.16292235, 0.12272133, ..., 0.03394827, 1.        ,
        0.11289456],
       [0.31355682, 0.05478139, 0.01369132, ..., 0.12493734, 0.11289456,
        1.        ]])

# **Similaridade De Pearson**

In [12]:
def calc_divisorPearson(u,v, mU, mV):
  sum_u = 0
  sum_v = 0
  itens_k = []

  itens_k = itens_comuns(u,v)
  for j in itens_k:
      sum_u += (u[j]-mU)**2
      sum_v += (v[j]-mV)**2
  return sum_u*sum_v

def similaridade_pearson(matriz):
  similaridade = np.zeros((row,row), float)
  for i in range(0, row):
    for j in range(i,row):
      if(i==j):
        similaridade[i,j] = 1
        continue
      parcial1 = 0
      parcial2 = 0
      a = matriz[i,0:]
      b = matriz[j,0:]
      itens_c = itens_comuns(a,b)

      if(len(itens_c) == 0):
        continue


      for k in itens_c:
        parcial1 += (matriz[i][k] - medias[i])*(matriz[j][k] - medias[j])

      parcial2 = math.sqrt(calc_divisorPearson(matriz[i,0:], matriz[j,0:], medias[i], medias[j]))


      similaridade[i][j] = 0 if math.isnan(parcial1/parcial2) else parcial1/parcial2
      similaridade[j][i] = similaridade[i][j]
  return similaridade


In [13]:
similar_b = similaridade_pearson(matriz)

/tmp/ipykernel_13797/1996271984.py:35: RuntimeWarning: invalid value encountered in scalar divide
  similaridade[i][j] = 0 if math.isnan(parcial1/parcial2) else parcial1/parcial2


In [14]:
pam_calc.append(similar_b)
similar_b

array([[ 1.        ,  0.35761538, -0.27342709, ...,  0.16257305,
        -0.36017554,  0.23907972],
       [ 0.35761538,  1.        ,  0.01868906, ...,  0.15506715,
        -0.03715266,  0.63258794],
       [-0.27342709,  0.01868906,  1.        , ...,  0.        ,
         0.4202462 , -1.        ],
       ...,
       [ 0.16257305,  0.15506715,  0.        , ...,  1.        ,
        -1.        ,  0.01924485],
       [-0.36017554, -0.03715266,  0.4202462 , ..., -1.        ,
         1.        ,  0.21800963],
       [ 0.23907972,  0.63258794, -1.        , ...,  0.01924485,
         0.21800963,  1.        ]])

# **Similaridade padrão**

In [15]:
def similaridade_ind(u,v):
  itens = []
  x = []
  y = []
  itens = itens_comuns(u,v)

  if(len(itens) == 0):
    return 0;
  for j in itens:
    x.append(u[j])
    y.append(v[j])
 # print(x, "\n", y)

  return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

def similaridade_padrao(matriz_notas):
  matriz_similar = np.zeros((row,row), float)
  for i in range(0,row):
    for j in range(i,row):
      if(i==j):
        matriz_similar[i,j] = 1
        continue
      sum = 0
      a = matriz[i,:]
      b = matriz[j,:]

      matriz_similar[i][j] = similaridade_ind(a,b)
      matriz_similar[j][i] = matriz_similar[i][j]
  return matriz_similar

In [16]:
similar_c = similaridade_padrao(matriz)

In [17]:
pam_calc.append(similar_c)


# **Previsão da classificação**

In [18]:
def viz_similar(m_similar, m_notas, item, user, min_notas, qtd):
  viz = pd.DataFrame({"u_viz": m_similar[user,0:],
                      "id_viz": np.arange(0, row, 1)})
  viz_f = []
  k = 0;
  viz = viz[viz["id_viz"] != user]
  viz = viz.sort_values(by = "u_viz", ascending = False)
  for i in viz["id_viz"]:
    if  M_qtd[i] > min_notas:
      if(m_notas[i, item] != 0 and k<qtd):
        viz_f.append(i)
        k +=1

  return viz_f

In [57]:
def prev_item(m_similar, m_notas,  item, user, min_notas, k):
  calc = 0;
  viz_s = []
  viz_s = viz_similar(m_similar, m_notas, item, user, min_notas, k)
  num = 0
  divisor = 0

  if len(viz_s) != k:
    return medias[user]

  for i in viz_s:
    num += m_similar[user, i]*(m_notas[i, item] - medias[i])
    divisor += abs(m_similar[user,i])
    #print(num, divisor)


  return medias[user] + num/divisor

# **Cálculo de MAE e RMSE**

In [56]:
def taxa_erro(m_similar, m_notas, gabarito, min_notas, viz = 5):

  df_comp = pd.DataFrame({"nota_prev": np.zeros(len(gabarito), float),
                          "nota_real": gabarito['ratings']})
  sum_MAE= 0
  sum_RMSE = 0
  n = len(gabarito)
  for i, info in gabarito.iterrows():
    user = info.id_user -1
    item = info.id_item -1
    #print("usuario", user, "\n item", item)
    y_p = prev_item(m_similar, m_notas, item, user, min_notas, viz,)
    sum_MAE += abs(info.ratings - y_p )
    sum_RMSE += (info.ratings - y_p)**2

  MAE = sum_MAE/n
  RMSE = math.sqrt(sum_RMSE/n)
  print(MAE, RMSE )

In [59]:
import pandas as pd
import numpy as np
import math


def taxa_erro(m_similar, m_notas, gabarito, min_notas, viz=5, threshold=3):
    df_comp = pd.DataFrame({"nota_prev": np.zeros(len(gabarito), float),
                            "nota_real": gabarito['ratings']})
    sum_MAE = 0
    sum_RMSE = 0
    n = len(gabarito)
    TP = FP = TN = FN = 0
    
    for i, info in gabarito.iterrows():
        user = info.id_user - 1
        item = info.id_item - 1
        y_p = prev_item(m_similar, m_notas, item, user, min_notas, viz)

        if not math.isnan(y_p):
            # Calculate errors for MAE and RMSE
            sum_MAE += abs(info.ratings - y_p)
            sum_RMSE += (info.ratings - y_p) ** 2
                
        
        # Binarize the ratings based on the threshold
        actual = info.ratings >= threshold
        predicted = y_p >= threshold
        
        # Calculate TP, FP, TN, FN
        if actual and predicted:
            TP += 1
        elif actual and not predicted:
            FN += 1
        elif not actual and predicted:
            FP += 1
        else:
            TN += 1
    
    MAE = sum_MAE / n
    RMSE = math.sqrt(sum_RMSE / n)
    
    # Calculate precision, recall, and F1-score
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
    print(f"MAE: {MAE}, RMSE: {RMSE}")
    print(f"Precision: {precision}, Recall: {recall}, F1-score: {f1_score}")
    return MAE, RMSE, precision, recall, f1_score


In [61]:
import matplotlib.pyplot as plt
# Valores dos eixos

nomes = ["Cosseno", "Pearson", "Padrão"]
data_validationC = pd.DataFrame({"Nome": nomes,
                                "Matrizes": pam_calc})
for i, info in data_validationC.iterrows():
  for j in range(5,31,5):
    MAE = []
    RMSE = []
    precisao = []
    recalla = []
    f1_scorea = []
    
    for k in range(0, 70, 20):
      print("Equação", info.Nome, "qtd de vizinhos:", j, "qtd minima de notas:", k)
      mae, rmse, precision, recall, f1_score = taxa_erro(info.Matrizes, matriz, X_val, k, j)
      MAE.append(mae)
      RMSE.append(rmse)
      precisao.append(precision)
      recalla.append(recall)
      f1_scorea.append(f1_score)
    print("\n")
    mean_mae = np.mean(MAE)
    mean_rmse = np.mean(RMSE)
    mean_precision = np.mean(precisao)
    mean_recall = np.mean(recalla)
    mean_f1_score = np.mean(f1_scorea)
    print("\n")
    print("Media MAE:", mean_mae, "Media RMSE:", mean_rmse, "Media Precision:", mean_precision, "Media Recall:", mean_recall, "Media F1-Score:", mean_f1_score)
    
    # Converter para numpy arrays para facilitar a manipulação
  MAE = np.array(MAE)
  RMSE = np.array(RMSE)
  precision = np.array(precision)
  recall = np.array(recall)
  f1_score = np.array(f1_score)

  print("\n")


Equação Cosseno qtd de vizinhos: 5 qtd minima de notas: 0
MAE: 0.7818703252096126, RMSE: 0.9909355721630524
Precision: 0.8964640040702111, Recall: 0.8491566265060241, F1-score: 0.872169285979458
Equação Cosseno qtd de vizinhos: 5 qtd minima de notas: 20


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Valores dos eixos
k_values = list(range(5, 31, 5))  # Quantidade de vizinhos
w_values = list(range(0, 70, 20))  # Quantidade mínima de avaliações

# Arrays de métricas (exemplo de dados)
MAE = [
    [0.8, 0.7, 0.6, 0.5],
    [0.75, 0.65, 0.55, 0.45],
    [0.7, 0.6, 0.5, 0.4],
    [0.65, 0.55, 0.45, 0.35],
    [0.6, 0.5, 0.4, 0.3]
]

RMSE = [
    [1.0, 0.9, 0.8, 0.7],
    [0.95, 0.85, 0.75, 0.65],
    [0.9, 0.8, 0.7, 0.6],
    [0.85, 0.75, 0.65, 0.55],
    [0.8, 0.7, 0.6, 0.5]
]

# Converter para numpy arrays para facilitar a manipulação
MAE = np.array(MAE)
RMSE = np.array(RMSE)

# Criar o gráfico de contorno para MAE
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
contour_mae = plt.contourf(k_values, w_values, MAE.T, cmap='viridis')
plt.colorbar(contour_mae)
plt.title('MAE')
plt.xlabel('Quantidade de Vizinhos (k)')
plt.ylabel('Quantidade Mínima de Avaliações (w)')

# Criar o gráfico de contorno para RMSE
plt.subplot(1, 2, 2)
contour_rmse = plt.contourf(k_values, w_values, RMSE.T, cmap='viridis')
plt.colorbar(contour_rmse)
plt.title('RMSE')
plt.xlabel('Quantidade de Vizinhos (k)')
plt.ylabel('Quantidade Mínima de Avaliações (w)')

plt.tight_layout()
plt.show()
